# Objective
The goal of this analysis is to utilize market basket analysis to determine what medications are most correlated with the medication abilfy. Creating a model that can determine medications that are commonly purchased together provides stakeholders with the ability to predict what medications patients may need and conduct further research for the cause of these correlations.

In [1]:
# Import Packages and Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [3]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [4]:
# Importing dataset

df = pd.read_csv(r"E:\Users\laisu\Downloads\medical_market_basket.csv",dtype={'locationid':np.int64})

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Presc01  7501 non-null   object
 1   Presc02  5747 non-null   object
 2   Presc03  4389 non-null   object
 3   Presc04  3345 non-null   object
 4   Presc05  2529 non-null   object
 5   Presc06  1864 non-null   object
 6   Presc07  1369 non-null   object
 7   Presc08  981 non-null    object
 8   Presc09  654 non-null    object
 9   Presc10  395 non-null    object
 10  Presc11  256 non-null    object
 11  Presc12  154 non-null    object
 12  Presc13  87 non-null     object
 13  Presc14  47 non-null     object
 14  Presc15  25 non-null     object
 15  Presc16  8 non-null      object
 16  Presc17  4 non-null      object
 17  Presc18  4 non-null      object
 18  Presc19  3 non-null      object
 19  Presc20  1 non-null      object
dtypes: object(20)
memory usage: 2.3+ MB


In [6]:
df.head()

,Presc01,Presc02,Presc03,Presc04,Presc05,Presc06,Presc07,Presc08,Presc09,Presc10,Presc11,Presc12,Presc13,Presc14,Presc15,Presc16,Presc17,Presc18,Presc19,Presc20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,amlodipine,albuterol aerosol,allopurinol,pantoprazole,lorazepam,omeprazole,mometasone,fluconozole,gabapentin,pravastatin,cialis,losartan,metoprolol succinate XL,sulfamethoxazole,abilify,spironolactone,albuterol HFA,levofloxacin,promethazine,glipizide
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,citalopram,benicar,amphetamine salt combo xr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning and Preparation
Before the analysis can be performed, our data needs to be cleaned and transformed as needed. Cleaning was conducted by detecting for missing values and duplicates. The data set provided also included many missing rows, these were removed. Missing values were also replaced with “false” and medications that were present resulted in “true.” To transform the data, I utilized the package transaction encoder to transform the data set into a logical data frame with each column representing an item and each row representing a transaction.

In [7]:
# Replacing NaN values with False, and making prescriptions that are present true

df = df[df['Presc01'].notna()]
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Presc01  7501 non-null   object
 1   Presc02  5747 non-null   object
 2   Presc03  4389 non-null   object
 3   Presc04  3345 non-null   object
 4   Presc05  2529 non-null   object
 5   Presc06  1864 non-null   object
 6   Presc07  1369 non-null   object
 7   Presc08  981 non-null    object
 8   Presc09  654 non-null    object
 9   Presc10  395 non-null    object
 10  Presc11  256 non-null    object
 11  Presc12  154 non-null    object
 12  Presc13  87 non-null     object
 13  Presc14  47 non-null     object
 14  Presc15  25 non-null     object
 15  Presc16  8 non-null      object
 16  Presc17  4 non-null      object
 17  Presc18  4 non-null      object
 18  Presc19  3 non-null      object
 19  Presc20  1 non-null      object
dtypes: object(20)
memory usage: 1.1+ MB


In [8]:
# Printing the first prescription

df.iloc[0]

Presc01                 amlodipine
Presc02          albuterol aerosol
Presc03                allopurinol
Presc04               pantoprazole
Presc05                  lorazepam
Presc06                 omeprazole
Presc07                 mometasone
Presc08                fluconozole
Presc09                 gabapentin
Presc10                pravastatin
Presc11                     cialis
Presc12                   losartan
Presc13    metoprolol succinate XL
Presc14           sulfamethoxazole
Presc15                    abilify
Presc16             spironolactone
Presc17              albuterol HFA
Presc18               levofloxacin
Presc19               promethazine
Presc20                  glipizide
Name: 0, dtype: object

In [9]:
df.shape

(7501, 20)

In [10]:
# Detectiing Duplicates

df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
7496    False
7497    False
7498     True
7499     True
7500    False
Length: 7501, dtype: bool

In [11]:
# Detecting Missing Values

df.isnull().sum()

Presc01       0
Presc02    1754
Presc03    3112
Presc04    4156
Presc05    4972
Presc06    5637
Presc07    6132
Presc08    6520
Presc09    6847
Presc10    7106
Presc11    7245
Presc12    7347
Presc13    7414
Presc14    7454
Presc15    7476
Presc16    7493
Presc17    7497
Presc18    7497
Presc19    7498
Presc20    7500
dtype: int64

In [12]:
# Transforming data with Transencoder turning python list into numpy array

trans = []
for i in range (0, 7501):
    trans.append([str(df.values[i,j]) for j in range(0, 20)])

In [13]:
TE = TransactionEncoder()
array = TE.fit(trans).transform(trans)

In [14]:
cleaned_df = pd.DataFrame(array, columns = TE.columns_)
cleaned_df

,Duloxetine,Premarin,Yaz,abilify,acetaminophen,actonel,albuterol HFA,albuterol aerosol,alendronate,allopurinol,...,trazodone HCI,triamcinolone Ace topical,triamterene,trimethoprim DS,valaciclovir,valsartan,venlafaxine XR,verapamil SR,viagra,zolpidem
0,False,False,False,True,False,False,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
for col in cleaned_df.columns:
    print(col)

Duloxetine
Premarin
Yaz
abilify
acetaminophen
actonel
albuterol HFA
albuterol aerosol
alendronate
allopurinol
alprazolam
amitriptyline
amlodipine
amoxicillin
amphetamine
amphetamine salt combo
amphetamine salt combo xr
atenolol
atorvastatin
azithromycin
benazepril
benicar
boniva
bupropion sr
carisoprodol
carvedilol
cefdinir
celebrex
celecoxib
cephalexin
cialis
ciprofloxacin
citalopram
clavulanate K+
clonazepam
clonidine HCI
clopidogrel
clotrimazole
codeine
crestor
cyclobenzaprine
cymbalta
dextroamphetamine XR
diazepam
diclofenac sodium
doxycycline hyclate
enalapril
escitalopram
esomeprazole
ezetimibe
fenofibrate
fexofenadine
finasteride
flovent hfa 110mcg inhaler
fluconozole
fluoxetine HCI
fluticasone
fluticasone nasal spray
folic acid
furosemide
gabapentin
glimepiride
glipizide
glyburide
hydrochlorothiazide
hydrocodone
hydrocortisone 2.5% cream
ibuprophen
isosorbide mononitrate
lansoprazole
lantus
levofloxacin
levothyroxine sodium
lisinopril
lorazepam
losartan
lovastatin
meloxicam
met

In [16]:
# Dropping NaN value

df_cleaned = cleaned_df.drop(['nan'], axis = 1)

In [17]:
for col in df_cleaned.columns:
    print(col)

Duloxetine
Premarin
Yaz
abilify
acetaminophen
actonel
albuterol HFA
albuterol aerosol
alendronate
allopurinol
alprazolam
amitriptyline
amlodipine
amoxicillin
amphetamine
amphetamine salt combo
amphetamine salt combo xr
atenolol
atorvastatin
azithromycin
benazepril
benicar
boniva
bupropion sr
carisoprodol
carvedilol
cefdinir
celebrex
celecoxib
cephalexin
cialis
ciprofloxacin
citalopram
clavulanate K+
clonazepam
clonidine HCI
clopidogrel
clotrimazole
codeine
crestor
cyclobenzaprine
cymbalta
dextroamphetamine XR
diazepam
diclofenac sodium
doxycycline hyclate
enalapril
escitalopram
esomeprazole
ezetimibe
fenofibrate
fexofenadine
finasteride
flovent hfa 110mcg inhaler
fluconozole
fluoxetine HCI
fluticasone
fluticasone nasal spray
folic acid
furosemide
gabapentin
glimepiride
glipizide
glyburide
hydrochlorothiazide
hydrocodone
hydrocortisone 2.5% cream
ibuprophen
isosorbide mononitrate
lansoprazole
lantus
levofloxacin
levothyroxine sodium
lisinopril
lorazepam
losartan
lovastatin
meloxicam
met

In [18]:
df_cleaned.shape

(7501, 119)

# Analysis
- Assoiciation Rules: Association rules are utilized to predict the probability of features being correlated to each other. To generate the association rules for our analysis, I utilized the apriori algorithm to calculate the frequencies of the items being purchased. I then utilized the association rules package to filter and prune for certain parameters.


- Filter by top 3 Rules: Lift indicates the measure of the likelihood of the consequent to occur when the antecedent is present. The support value is the measure of the frequency an itemset occurs within the dataset. The confidence is the measure of likelihood an itemset will occur if another itemset is present. By filtering the association rules with a lift of at least 1.9, support level of at least 0.02, and a confidence level of at least 0.34, we can identify the top three rules of our association table. 

In [19]:
# Establishing Association Rules with Apriori Algorithm

a_rules = apriori(df_cleaned, min_support = 0.02, use_colnames = True)
a_rules.head()

,support,itemsets
0,0.046794,(Premarin)
1,0.238368,(abilify)
2,0.020397,(albuterol aerosol)
3,0.033329,(allopurinol)
4,0.079323,(alprazolam)


In [20]:
# Association Rules Table

ass_r = association_rules(a_rules, metric = 'lift', min_threshold = 1)
ass_r

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(amlodipine),(abilify),0.071457,0.238368,0.023597,0.330224,1.385352,0.006564,1.137144,0.299568
1,(abilify),(amlodipine),0.238368,0.071457,0.023597,0.098993,1.385352,0.006564,1.030562,0.365218
2,(amphetamine salt combo),(abilify),0.068391,0.238368,0.024397,0.356725,1.496530,0.008095,1.183991,0.356144
3,(abilify),(amphetamine salt combo),0.238368,0.068391,0.024397,0.102349,1.496530,0.008095,1.037830,0.435627
4,(amphetamine salt combo xr),(abilify),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815,0.193648
...,...,...,...,...,...,...,...,...,...,...
89,(diazepam),(metoprolol),0.163845,0.095321,0.022930,0.139951,1.468215,0.007312,1.051893,0.381390
90,(glyburide),(doxycycline hyclate),0.170911,0.095054,0.020131,0.117785,1.239135,0.003885,1.025766,0.232768
91,(doxycycline hyclate),(glyburide),0.095054,0.170911,0.020131,0.211781,1.239135,0.003885,1.051852,0.213256
92,(glyburide),(losartan),0.170911,0.132116,0.028530,0.166927,1.263488,0.005950,1.041786,0.251529


In [21]:
# Filtering for top 3 rules

ass_r[ (ass_r['lift'] >= 1.9) &
    (ass_r['confidence'] >= 0.34) &
     (ass_r['support'] >=0.02)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
30,(metformin),(abilify),0.050527,0.238368,0.023064,0.456464,1.914955,0.011020,1.401255,0.503221
72,(glipizide),(carvedilol),0.065858,0.174110,0.022930,0.348178,1.999758,0.011464,1.267048,0.535186
74,(lisinopril),(carvedilol),0.098254,0.174110,0.039195,0.398915,2.291162,0.022088,1.373997,0.624943


In [22]:
# Filtering for Correlation to Abilify

ant_df = ass_r[ass_r['antecedents'] == {'abilify'}].sort_values(by=['lift'], ascending = False)
con_df = ass_r[ass_r['consequents'] == {'abilify'}].sort_values(by=['lift'], ascending = False)
ant_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
31,(abilify),(metformin),0.238368,0.050527,0.023064,0.096756,1.914955,0.011020,1.051182,0.627330
25,(abilify),(glipizide),0.238368,0.065858,0.027596,0.115772,1.757904,0.011898,1.056449,0.566075
29,(abilify),(lisinopril),0.238368,0.098254,0.040928,0.171700,1.747522,0.017507,1.088672,0.561638
23,(abilify),(fenofibrate),0.238368,0.051060,0.020131,0.084452,1.653978,0.007960,1.036472,0.519145
15,(abilify),(clopidogrel),0.238368,0.059992,0.022797,0.095638,1.594172,0.008497,1.039415,0.489364
33,(abilify),(metoprolol),0.238368,0.095321,0.035729,0.149888,1.572463,0.013007,1.064189,0.477993
7,(abilify),(atorvastatin),0.238368,0.129583,0.047994,0.201342,1.553774,0.017105,1.089850,0.467950
3,(abilify),(amphetamine salt combo),0.238368,0.068391,0.024397,0.102349,1.496530,0.008095,1.037830,0.435627
21,(abilify),(doxycycline hyclate),0.238368,0.095054,0.033729,0.141499,1.488616,0.011071,1.054100,0.430963
35,(abilify),(naproxen),0.238368,0.058526,0.020131,0.084452,1.442993,0.006180,1.028318,0.403076


In [23]:
con_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
30,(metformin),(abilify),0.050527,0.238368,0.023064,0.456464,1.914955,0.011020,1.401255,0.503221
24,(glipizide),(abilify),0.065858,0.238368,0.027596,0.419028,1.757904,0.011898,1.310962,0.461536
28,(lisinopril),(abilify),0.098254,0.238368,0.040928,0.416554,1.747522,0.017507,1.305401,0.474369
22,(fenofibrate),(abilify),0.051060,0.238368,0.020131,0.394256,1.653978,0.007960,1.257349,0.416672
14,(clopidogrel),(abilify),0.059992,0.238368,0.022797,0.380000,1.594172,0.008497,1.228438,0.396502
32,(metoprolol),(abilify),0.095321,0.238368,0.035729,0.374825,1.572463,0.013007,1.218270,0.402413
6,(atorvastatin),(abilify),0.129583,0.238368,0.047994,0.370370,1.553774,0.017105,1.209650,0.409465
2,(amphetamine salt combo),(abilify),0.068391,0.238368,0.024397,0.356725,1.496530,0.008095,1.183991,0.356144
20,(doxycycline hyclate),(abilify),0.095054,0.238368,0.033729,0.354839,1.488616,0.011071,1.180529,0.362712
34,(naproxen),(abilify),0.058526,0.238368,0.020131,0.343964,1.442993,0.006180,1.160960,0.326080


In [24]:
cleaned_df.to_csv (r"E:\Users\laisu\Documents\Market_basket.csv")

# Results
To answer our research question of determining the most correlated medications with Abilify, we need to further filter our analysis. To accomplish this, I filtered for the medication Abilify as either the antecedent or consequent and sorted by lift. I chose lift as the main sorting measurement as values of lift greater than one have an increased correlation. Evaluating the medication with the highest lift value in relationship to Abilify, we can see that the medication is metformin. The lift of this itemset is 1.91, indicating a high correlation that a person will purchase metformin in addition to abilify. The support of this itemset is 0.023, indicating the itemset frequency within this dataset is 2.3 percent. The confidence of this itemset is the proportion of prescriptions that include the itemset divided by the proportion of just the antecedent. Analyzing abilify as the antecedent and the consequent, the support is higher at 0.46, or 46 percent. 

# Recommendations
The results of our research question are that customers within this dataset that purchase abilify are likely to also purchase metformin. I would recommend stakeholders to further research the average costs and necessity for these medications to be prescribed together. I would recommend stakeholders to interview prescribing providers for their indication for these medications to be taken together. If the reasoning is to prevent weight gain from abilify, are there alternative methods of weight gain prevention that can be implemented for cost savings to the patient. 